# Testing & Inference

## Introduction

**Learning Objectives**
- Explaining the difference between **training**, **testing** and **inference**  
- Describing how a trained neural network makes predictions on unseen data  
- Computing and interpreting **accuracy** as a measure of model performance  
- Understanding the role of **epochs** in *evaluating* learning progress  
- Assessing whether a model is *generalising* or *overfitting* using test data  

By the end of this lesson, you will have a clear understanding of how a trained neural network is evaluated and used in practice. Building on the previous sections, we will continue focusing on the **multi-layer perceptron (MLP)**, to examine how model performance is assessed once out training has been completed.

## What is testing?

Using handwritten digit recognition as a running example, we will explore how a trained network makes predictions on previously unseen data and how these predictions are compared to known labels. In particular, we introduce the **accuracy** metric - a simple and commonly used score for quantifying how well a model performs.

KEY TERMS BOX

**Accuracy** is a *performance metric* that measures the **proportion of correct predictions** made by a model. 

It is defined as the number of correct predictions divided by the total number of predictions:

$$
\text{accuracy} = \frac{\text{number of correct predictions}}{\text{total number of predictions}}
$$

The schematic below illustrates this idea for the handwritten digit classification task.

![Fig. 7: Testing unlabelled data using a trained model.](./images/Fig_testing.png)

The images used for this evaluation are drawn from the labelled data available to us, but crucially are *not* the same images used during training. In practice, the full labelled dataset is typically split into two parts. For instance: around 80% of the samples may be used for training, while the remaining 20% are reserved for testing.

The **test set** contains data that the network has never encountered during training, and this separation is essential. If we were to evaluate the model only on training data, we would have no reliable way of knowing whether it has genuinely learned the task or has merely memorised the examples it was shown, without the ability to generalise on unlabelled data. This issue is closely related to the problem of **overfitting**. More information can be found in the [further reading](./further-reading.html#testing---inference-resources) section of this lesson.

In practice, model performance on the test set is often monitored throughout the training process, rather than just at the very end. This allows us to observe how a model's performance *evolves* over time: both on the training data used to update the network’s parameters and on unseen data used purely for model evaluation. Tracking these trends provides valuable insight into whether the model is learning effectively, or beginning to overfit.

While we do not explore monitoring training progress in extensive detail here, it is worth noting that a further distinction is often made between **validation** and **test** sets. A *validation set* may be used to tune hyperparameters such as the learning rate, while the *test set* is reserved for final performance assessment.

Finally, the term **inference** is typically used to describe the situation in which a trained model is applied to new, unseen data for which no ground truth labels are provided. In this setting, the model’s predictions are taken as its best estimate of the correct output.

## The AND gate function

Let's set out another worked example, in this section of the lesson. We begin by defining a simple neural network with a single hidden layer. The computation performed by this layer can be written as:

$$
y = \tanh(Wx + b)
$$

Here, $W$ denotes a matrix of weights, $b$ is a vector of bias terms, and $x$ is a vector of input values. As discussed earlier, the weights and biases of the network are learned during training through **gradient descent**.

Our goal is to construct a layer of neurons that can learn the behaviour of an **AND gate**. The term 'AND gate' refers to a basic logical operation that outputs 1 only when *both* of its inputs are 1. In all other cases, the output is 0.

The truth table for an AND gate is shown below:

| first input ($x_1$) | second input ($x_2$) | output ($y$) |
|---|---|---|
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

In this setup, the columns $x_1$ and $x_2$ form the inputs to the network, while the column $y$ represents the desired output. We can collect these values into matrix form as follows:

$$
X =
\begin{bmatrix}
0 & 0 \\
0 & 1 \\
1 & 0 \\
1 & 1
\end{bmatrix}
$$

$$
y =
\begin{bmatrix}
0 \\
0 \\
0 \\
1
\end{bmatrix}
$$

Each row of $X$ corresponds to one possible input combination for the AND gate, and the matching entry in $y$ gives the correct output for that input.

KEY TERMS BOX

An **epoch** is one complete pass through the entire training dataset during the learning process. During each epoch, the model processes all training samples, updates its parameters based on the computed loss, gradually improving its performance.

KEY TERMS BOX

The **learning rate** is a *hyperparameter* that controls **how large a step the model takes** when updating its parameters during training. It determines *how quickly the model learns*, with larger values leading to faster but potentially unstable updates, and smaller values leading to slower but more stable learning.

In the code block below, let's train a simple neural network to predict the correct output $y$, given the inputs $x1$ and $x2$. 

The code defines variables for `epochs` and `learning_rate`, so these can be easily adjusted, and the code cell re-run. A prediction and loss calculation are performed at each epoch, and the code uses a conditional statement and the modulus operator `%` to print the loss and epoch number, every 500 epochs.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

x_train = np.array(
    [[0, 0],
    [0, 1],
    [1, 0],
    [1, 1]], dtype=np.float32)

y_train = np.array([[0], [0], [0], [1]], dtype=np.float32)

# Convert the numpy arrays to PyTorch tensors:
x_train = torch.from_numpy(x_train) 
y_train = torch.from_numpy(y_train)

# The number of epochs is the number of times the training data is passed through the single-layer neural network:
epochs = 5000

# The learning rate is the step size for the gradient descent:
learning_rate = 0.1

# Number of features - 2 in this case
input_size = len(x_train[0])

# Number of outputs - 1 in this case (on/off)
output_size = 1

losses = []
w = torch.randn(input_size, output_size, requires_grad=True)
b = torch.randn(1, 1, requires_grad=True)

for e in range(epochs):

    # Calculate the output of the neuron:
    y_pred = torch.matmul(x_train, w) + b

    # Apply a sigmoid activation function to the output of the neuron:
    y_pred = torch.sigmoid(y_pred)

    # Calculate the loss:
    loss = (y_pred - y_train).pow(2).sum() # MSE loss

    # Print the loss every 500 epochs
    if e % 500 == 0:
        print(f'Epoch: {e} | loss: {loss.item():.4f}')
    losses.append(loss.item())

    # Calculate the gradients of the loss with respect to the weights and biases:
    loss.backward()

    # Update the weights and biases:
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad

        # Zero the gradients. Otherwise, they will accumulate:
        w.grad.zero_()
        b.grad.zero_()

PRACTICE EXERCISE

In the code block given above, we train a neural network to learn the AND gate function with a **learning rate of 0.1**.

- Copy the AND gate training code from the cell above into the cell below

- Change `learning_rate = 0.1` to `learning_rate = 0.01`

- You may wish to reduce `epochs` to `2000` to speed up your experiments

- Run the code and observe the final loss values and the shape of the loss curve.

- Try again, setting different values for the `learning_rate` variable, such as `0.5` or `1.0`

**Questions**:

1. Does the model still learn the AND gate correctly with a very small learning rate (e.g. 0.01)?

2. What happens to the loss curve when the learning rate is too high (e.g. 1.0 or 2.0)?

3. Which learning rate value gives the best balance between speed of learning and stability?

In [ ]:
# Download this notebook and write your code here.

###     

Once you have attempted the practice exercise, let's try to answer those questions, visually.

As we saw in the previous section, as the model is trained, the loss decreases. As visualised below:

In [ ]:
# Plot the loss over time:
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over time')

plt.show()
# Ṭhis shows that the loss is decreasing over time, which means that the neural network is learning.

Now that we have a trained model, we can assess its *performance*. The block of code below inspects how well our trained model solves the task:

In [ ]:
# Compare the real labels of the AND gate to the predicted labels from the perceptron:

# Make two subplots, side by side:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot the original data:
ax1.scatter(x_train[:, 0], x_train[:,1], c=y_train.numpy(), s=200)
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')

# Plot the predicted data:
ax2.scatter(x_train[:, 0], x_train[:,1], c=y_pred.detach().numpy(), s=200)
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')

# Add a title to each subplot:
ax1.set_title('Original data')
ax2.set_title('Predicted data')

# Add a title to the figure:
fig.suptitle('A layer of neurons learning an AND gate')

# Make a manual legend:
from matplotlib.lines import Line2D
from matplotlib import cm
import matplotlib.colors as colors

norm = colors.Normalize(vmin=0, vmax=1)
rgba_color_0 = cm.viridis(norm(0.01)) 
rgba_color_1 = cm.viridis(norm(.99)) 
line1 = Line2D([], [], color="white", marker='o', markerfacecolor=rgba_color_0, markersize=15)
line2 = Line2D([], [], color="white", marker='o', markerfacecolor=rgba_color_1, markersize=15)

plt.legend((line1, line2), ('OFF', 'ON'), numpoints=1, loc=1, bbox_to_anchor=(1.3, 1.01))
plt.tight_layout()
plt.show()

As can be seen from the plot, the predictions $y$ of the simple network match the ground-truth data for the same inputs $x1, x2$.

NOTE BOX

This problem is particularly interesting in the context of neural networks because it cannot be solved using neurons whose activation functions is linear: we need the activation functions to be non-linear to solve this task. As we explored in the previous section.

## MNIST classification

In this example, we will use **PyTorch** to implement a multi-layer perceptron (MLP) for classifying the **MNIST dataset**. As mentioned previously in this lesson, MNIST consists of greyscale images of handwritten digits and is commonly used as a benchmark dataset for developing and evaluating machine learning models in image analysis.

When working with PyTorch, we do not need to implement every component of the network and training process from scratch. The framework provides a range of high-level modules and utilities that simplify the design, training and evaluation of neural networks. In particular, the `nn` module allows us to define network layers in a clear and structured way.

Using the `Sequential` class from the `nn` module, we can construct a neural network as an *ordered sequence of fully-connected layers*. The same module can also provide ready-made components for defining the loss function and the optimisation algorithm, allowing us to focus on the overall structure and behaviour of the model rather than low-level implementation details.

In [ ]:
# Use the nn module to create an MLP to classify MNIST, with 3 layers:

# Import the nn module from PyTorch
import torch.nn as nn

# Define the MLP class:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # The forward pass is the sequence of operations 
        # that the input x goes through in the network. 
        # In this case, the input goes through 3 linear layers,
        # where a ReLU activation function is applied to the 
        # output of the first two layers, before the output is
        # passed through a softmax activation function:
        out = nn.Sequential(
            # flatten the image:
            nn.Flatten(),
            # apply the first linear layer:
            self.linear1,
            # apply the ReLU activation function:
            nn.ReLU(),
            # apply the second linear layer:
            self.linear2,
            # apply the ReLU activation function:
            nn.ReLU(),
            # apply the third linear layer:
            self.linear3,
            # apply the softmax activation function. 
            # This is necessary because the output 
            # of the network is a probability distribution, 
            # and we want the the most probable class to be 
            # 1 and the rest to be 0:
            nn.Softmax(dim=1)
        )(x)
        return out


The figure below shows how a trained model processes an image, where each pixel is treated as an individual input.

![Fig. 7: Showing the pixel-wise computing of an image through a trained model.](./images/Fig_pixels.png)

Now that we have defined our MLP, we are ready to *download* the MNIST dataset. Rather like sklearn, PyTorch has a built-in function that allows us to download the MNIST dataset, and load it into an object called a **data loader**. Data loaders include convenient *methods* facilitating data to be loaded in, in batches, and applying transformations to it. We will use the data loader to load the data into batches of **64 images**, and to then normalise these.

In [ ]:
from torchvision import datasets, transforms

# Define a training data loader

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                     transform=transforms.Compose([
                          transforms.ToTensor(),    # convert the image to a tensor
                          transforms.Normalize((0.1307,), (0.3081,)) # normalize the image
                        ])),
    batch_size=64, shuffle=True)

# Define a test data loader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                        ])),
    batch_size=1000, shuffle=True)

# Define a function to plot the images
def plot_images(images, labels):
    fig, axes = plt.subplots(4, 4)
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(28, 28), cmap='binary')
        ax.set(xticks=[], yticks=[], xlabel=int(labels[i]))
        # Make a white background for the text
        ax.xaxis.label.set_color('white')
        # Increase the vertical distance between the subplots:
        fig.subplots_adjust(hspace=0.5)
    plt.show()


# Plot a few of the MNIST images with their labels:
images, labels = next(iter(train_loader))
plot_images(images, labels)

Finally, let's define two functions to *train* and *test* the model:

In [ ]:
# Define a function to train the model:

def train(model, device, train_loader, optimizer, epoch):
    model.train() # set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # send the data to the device
        optimizer.zero_grad() # zero the gradients
        output = model(data) # forward pass
        loss = F.nll_loss(output, target) # calculate the loss
        loss.backward() # backward pass
        optimizer.step() # update the weights

        # Print the training loss
        if batch_idx % 500 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Define a function to test the model
def test(model, device, test_loader):
    model.eval() # set the model to evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad(): # don't calculate the gradients
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) # send the data to the device
            output = model(data) # forward pass
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up the batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability

            # Check if the prediction is correct
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Calculate the average loss and accuracy
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))



We can now run our training loop. For demonstrative purposes, we will train it for a few epochs, only. In a real-world application, however, we would typically train the model over several hundreds of epochs, or more.

In [ ]:
# Run the training
# Imports
import torch.optim as optim
import torch.nn.functional as F

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an instance of the MLP class
model = MLP(input_size=784, hidden_size=512, output_size=10).to(device)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train the model for 5 epochs
for epoch in range(5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Next, let's see how our trained model performs on the test set.

In [ ]:
# We can now use the model to make predictions on the test set:

# get the first batch of test images
images, labels = next(iter(test_loader))

# plot the images
plot_images(images, labels)

# get the model predictions
preds = model(images.to(device))

# get the index of the max log-probability
preds = preds.argmax(dim=1, keepdim=True)

# print the predictions

predicted_labels_list = preds.squeeze().tolist() 
print('Predicted labels:', predicted_labels_list[:16])

# print the actual labels
print('Actual labels:   ', labels[:16].tolist())

# Accuracy
accuracy = 100*(sum(np.array(predicted_labels_list) == np.array(labels.tolist())) / len(predicted_labels_list))
print(f'Accuracy: {accuracy} %')

With an accuracy score of around 80%, the model is performing very respectably. Training over only a few epochs led to the model only misclassifying one handwritten number - a 6, as a 1.

We have now trained and evaluated a neural network for the classic handwritten digit recognition task, and the MNIST dataset.

NOTE BOX

The scores are calculated after we write the notebooks, so we're just hoping the run gives a score similar to the ones we normally expect. Don't blame us entirely if it's wildly off...

## Summary

In this lesson, we examined how a trained neural network is evaluated and used beyond the training phase. Using a **multi-layer perceptron** applied to handwritten digit recognition, we explored how model predictions are generated through a *forward pass*, how performance is *quantified* using **accuracy**, and why **evaluation** *must be carried out on data that the model has not seen* during training. We also introduced the concept of **inference**, where a trained model is applied to new, unlabelled data. Together, these ideas complete the end-to-end picture of training, testing and deploying a simple neural network, and provide a foundation for working with more advanced models.